# Data collection and preprocessing

**Two municipalities will merge Jan 1, 2025. Not sure when this will be reflected in the data files.**

All imports and general definitions used throughout the notebook are here.

In [1]:
import requests
import pandas as pd
import geopandas as gpd
import helpers

dirty_dir = "unprocessed_datasets/"
clean_dir = "preprocessed_datasets/"

maps = "zip://" + dirty_dir + "land_survey/TietoaKuntajaosta_2024_10k.zip"
municipality_map = gpd.read_file(maps, layer="SuomenKuntajako_2024_10k", engine="pyogrio")

# See also dataframe "mappings" defined in "0. Mappings"

## 0. Mappings

This dataset contains information municipality (kunta) and region (maakunta) names in different languages, and the link between them.

- Data source: [Statistics Finland](https://stat.fi/index_en.html)
- License: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/deed.en)
- Links:
    - [Data](https://stat.fi/en/luokitukset/kunta/kunta_1_20240101)

This code builds a dataset which is used to:
- map the names of the municipalities and regions in the data to their respective codes
- map municipality and region codes to (English) names

If needed, a helpers function can be created to map Finnish names to Swedish or English, or vice versa.

Names for municipalities and regions:
- Finnish: https://stat.fi/fi/luokitukset/corrmaps/kunta_1_20240101%23maakunta_1_20240101
- Swedish: https://stat.fi/sv/luokitukset/corrmaps/kunta_1_20240101%23maakunta_1_20240101
- English: https://stat.fi/en/luokitukset/corrmaps/kunta_1_20240101%23maakunta_1_20240101

In [2]:
# Read in the data in different languages

statfin1_fi = pd.read_csv(dirty_dir + "statistics_finland/kunta_1_20240101#maakunta_1_20240101_fi.csv", header=0, sep=";")
statfin1_sv = pd.read_csv(dirty_dir + "statistics_finland/kunta_1_20240101#maakunta_1_20240101_sv.csv", header=0, sep=";")
statfin1_en = pd.read_csv(dirty_dir + "statistics_finland/kunta_1_20240101#maakunta_1_20240101_en.csv", header=0, sep=";")

# Rename columns holding names

statfin1_fi = statfin1_fi.rename(columns={'sourceName': 'municipality_fi', 'targetName': 'region_fi'})
statfin1_sv = statfin1_sv.rename(columns={'sourceName': 'municipality_sv', 'targetName': 'region_sv'})
statfin1_en = statfin1_en.rename(columns={'sourceName': 'municipality_en', 'targetName': 'region_en'})

# Merge the data

statfin1_all = statfin1_fi.merge(statfin1_sv, on=['sourceCode', 'targetCode']).merge(statfin1_en, on=['sourceCode', 'targetCode'])

# Rename columns

statfin1_all = statfin1_all.rename(columns={'sourceCode': 'municipality_id', 'targetCode': 'region_id'})

# Drop extra columns

statfin1_all = statfin1_all.drop(columns=[col for col in statfin1_all.columns if col.startswith(('distribution', 'Unnamed'))])

# Remove quotes from the data

statfin1_all['municipality_id'] = statfin1_all['municipality_id'].str.strip().str.replace('"', '').str.replace("'", '')
statfin1_all['region_id'] = statfin1_all['region_id'].str.strip().str.replace('"', '').str.replace("'", '')

# Define the data types

statfin1_all['municipality_id'] = pd.to_numeric(statfin1_all['municipality_id'])
statfin1_all['region_id'] = pd.to_numeric(statfin1_all['region_id'])

statfin1_all.head(5)

,municipality_id,municipality_fi,region_id,region_fi,municipality_sv,region_sv,municipality_en,region_en
0,20,Akaa,6,Pirkanmaa,Ackas,Birkaland,Akaa,Pirkanmaa
1,5,Alajärvi,14,Etelä-Pohjanmaa,Alajärvi,Södra Österbotten,Alajärvi,South Ostrobothnia
2,9,Alavieska,17,Pohjois-Pohjanmaa,Alavieska,Norra Österbotten,Alavieska,North Ostrobothnia
3,10,Alavus,14,Etelä-Pohjanmaa,Alavo,Södra Österbotten,Alavus,South Ostrobothnia
4,16,Asikkala,7,Päijät-Häme,Asikkala,Päijänne-Tavastland,Asikkala,Päijät-Häme


Future development ideas:
- Create a dictionary, store as a JSON file, and load it in the notebook

In [3]:
# Write clean data to a folder containing preprocessed files

helpers.save_dataset(clean_dir + "municipality_and_region_information.csv", statfin1_all)

If you skip the code generating the mapping dataset, you can load it here. The code below expects to find it.

In [4]:
mappings = helpers.load_dataset(clean_dir + "municipality_and_region_information.csv")

## 1a. All vehicles

**This dataset is currently not used in the analysis and the preprocessing is not complete so it's safe to skip this section.**

This dataset contains information about every registered vehicle in Finland.

- Data source: [Finnish Transport and Communications Agency, Traficom](https://www.traficom.fi/en)
- License: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/deed.en)
- Links:
    - [Data (.csv)](https://tieto.traficom.fi/en/datatraficom/open-data?toggle=Open%20data%20for%20vehicles)
    - [Definitions (.xlsx)](https://tieto.traficom.fi/sites/default/files/media/file/Ajoneuvotiedot_avoin_data_luokitukset_24.01.2023.xlsx)

Manual preprocessing steps:

1. Go to https://tieto.traficom.fi/en/datatraficom/open-data?toggle=Open%20data%20for%20vehicles
2. Download the ZIP file
3. Unzip ("datasets/traficom/" used in this notebook)
4. Check the file name matches the one in the next cell

Both URLs or file paths are accepted by pd.read_csv(), but this dataset is almost a GB --> use a file path!

Ignore the warning about mixed types in columns. The next step is to clean up the data and define column data types.

In [5]:
# Do not try to open this one directly from an URL

datafile = dirty_dir + "traficom/Ajoneuvojen_avoin_data_5_25.csv"
traficom1 = pd.read_csv(datafile, header=0, sep=";", encoding="ISO-8859-1")

print(traficom1.shape)
traficom1.head(5)

/var/folders/5h/zh1_sxmx7wv1zb3rdkjycj4h0000gn/T/ipykernel_2570/2450349567.py:4: DtypeWarning: Columns (18,27,32) have mixed types. Specify dtype option on import or set low_memory=False.
  traficom1 = pd.read_csv(datafile, header=0, sep=";", encoding="ISO-8859-1")


(5373623, 41)


,ajoneuvoluokka,ensirekisterointipvm,ajoneuvoryhma,ajoneuvonkaytto,variantti,versio,kayttoonottopvm,vari,ovienLukumaara,korityyppi,...,tyyppihyvaksyntanro,yksittaisKayttovoima,kunta,NEDC_Co2,NEDC2_Co2,WLTP_Co2,WLTP2_Co2,matkamittarilukema,valmistenumero2,jarnro
0,L3,2003-06-10,109.0,1.0,NaN,NaN,20030610,0,NaN,NaN,...,NaN,1.0,297.0,NaN,NaN,NaN,NaN,NaN,NaN,1
1,MUU,NaN,21.0,1.0,NaN,NaN,19670000,NaN,NaN,NaN,...,NaN,1.0,91.0,NaN,NaN,NaN,NaN,NaN,NaN,2
2,MUU,1976-09-01,13.0,1.0,NaN,NaN,19760000,NaN,NaN,NaN,...,NaN,NaN,475.0,NaN,NaN,NaN,NaN,NaN,NaN,3
3,M1,1984-07-09,NaN,1.0,NaN,NaN,19840000,1,NaN,NaN,...,NaN,1.0,740.0,NaN,NaN,NaN,NaN,NaN,NaN,4
4,MUU,1983-09-22,13.0,1.0,NaN,NaN,19830000,NaN,NaN,NaN,...,NaN,NaN,734.0,NaN,NaN,NaN,NaN,NaN,NaN,5


In [6]:
# Vehicle classes (ajoneuvoluokka) "M1" and "M1G" mean passenger cars
# Removing everything else brings the row count to less than 3 Million

traficom1 = traficom1[traficom1["ajoneuvoluokka"].isin(["M1", "M1G"])]

# TODO: More data wrangling

print(traficom1.shape)
traficom1.head(5)

(2836469, 41)


,ajoneuvoluokka,ensirekisterointipvm,ajoneuvoryhma,ajoneuvonkaytto,variantti,versio,kayttoonottopvm,vari,ovienLukumaara,korityyppi,...,tyyppihyvaksyntanro,yksittaisKayttovoima,kunta,NEDC_Co2,NEDC2_Co2,WLTP_Co2,WLTP2_Co2,matkamittarilukema,valmistenumero2,jarnro
3,M1,1984-07-09,NaN,1.0,NaN,NaN,19840000,1,NaN,NaN,...,NaN,1.0,740.0,NaN,NaN,NaN,NaN,NaN,NaN,4
6,M1,1990-05-08,NaN,5.0,NaN,NaN,19900508,9,NaN,NaN,...,NaN,1.0,91.0,NaN,NaN,NaN,NaN,NaN,VF7XBEY000,7
7,M1,2003-10-02,NaN,1.0,1,4,20031002,6,NaN,AA,...,e6*2001/116*0091*00,1.0,837.0,190.0,NaN,NaN,NaN,284104.0,JHMCL75403,8
8,M1,2006-03-17,NaN,1.0,ZZE121(W),ZZE121L-DWMNKW(1G),20060317,Y,NaN,AC,...,e11*2001/116*0181*06,1.0,989.0,172.0,NaN,NaN,NaN,155944.0,NMTEZ20E80,9
9,M1,2007-01-05,NaN,1.0,ALA30(WT),ALA30L-AWFGYW(1B),20070105,2,NaN,AF,...,e6*2001/116*0105*01,2.0,694.0,173.0,NaN,NaN,NaN,2692651.0,JTMBA31V10,10


In [7]:
# Write clean data to a folder containing preprocessed files

helpers.save_dataset(clean_dir + "all_vehicles.csv", traficom1)

## 1b. Vehicles in traffic by region (maakunta) per quarter

This dataset contains statistics about all vehicle classes in traffic in Finland over time.

- Data source: [Finnish Transport and Communications Agency, Traficom](https://www.traficom.fi/en)
- License: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/deed.en)
- Link: https://trafi2.stat.fi/PXWeb/pxweb/en/TraFi/TraFi__Liikennekaytossa_olevat_ajoneuvot/?tablelist=true

Manual preprocessing steps:

1. Go to https://trafi2.stat.fi/PXWeb/pxweb/en/TraFi/TraFi__Liikennekaytossa_olevat_ajoneuvot/040_kanta_tau_104.px/
2. Select fields:
    - Region: all but "MAINLAND FINLAND", "Foreign countries", "Unknown"
    - Vehicle class: Only "Passenger cars"
    - Driving power: All
    - Quarter: All
3. Press "Show table"
4. Click "Pivot manual" and move all columns to rows in the same order as they appeared, then click "Complete"
5. Click "Edit and Calculate", then "Split time variable"
6. Click "Save result as...", then "Comma delimited with heading" and press "Save", choose "datasets/traficom" as the folder
7. Check the file name matches the one in the next cell

This saved query should result in the same outcome: https://trafi2.stat.fi:443/PXWeb/sq/b827cb63-4931-4c40-b399-ad36384f2b69

In [8]:
datafile = f"{dirty_dir}traficom/ajkanta.csv"
traficom2 = pd.read_csv(datafile, header=1, sep=",", encoding="ISO-8859-1")

traficom2.head(5)

,Region,Vehicle class,Driving power,year,quarter,Vehicles in traffic
0,Uusimaa,Passenger cars,Total,2008,Quarter 1,636847
1,Uusimaa,Passenger cars,Total,2008,Quarter 2,646364
2,Uusimaa,Passenger cars,Total,2008,Quarter 3,643667
3,Uusimaa,Passenger cars,Total,2008,Quarter 4,632722
4,Uusimaa,Passenger cars,Total,2009,Quarter 1,633425


In [9]:
# Replace missing values with zeroes

traficom2.replace("-", 0, inplace=True)
traficom2.replace(".", 0, inplace=True)

# Replace "Quarter x" with the numerical value

traficom2["quarter"] = traficom2["quarter"].replace("Quarter 1", "1")
traficom2["quarter"] = traficom2["quarter"].replace("Quarter 2", "2")
traficom2["quarter"] = traficom2["quarter"].replace("Quarter 3", "3")
traficom2["quarter"] = traficom2["quarter"].replace("Quarter 4", "4")
traficom2["quarter"] = pd.to_numeric(traficom2["quarter"])

# Convert column "Vehicles in traffic" to numeric

traficom2["Vehicles in traffic"] = pd.to_numeric(traficom2["Vehicles in traffic"])

# Reclassify the data in column "Driving power", then group the rows for new totals

traficom2["Driving power"] = traficom2["Driving power"].replace({
    "Petrol/Electricity (plug-in hybrid)": "Petrol/Diesel + electricity",
    "Diesel/Electricity (plug-in hybrid)": "Petrol/Diesel + electricity",
    "Hydrogen": "Other",
    "Gas": "Other",
    "Fuel oil": "Other",
    "Biodiesel": "Other",
    "Liquefied petroleum gas (LPG)": "Other",
    "Light fuel oil (kerosene)": "Other",
    "Petrol/wood": "Other",
    "Petrol + light fuel oil (kerosene)": "Other",
    "Ethanol": "Other",
    "Petrol/CNG": "Petrol/Diesel + CNG",
    "Diesel/CNG": "Petrol/Diesel + CNG",
    "Petrol/Ethanol": "Petrol + ethanol",
    "Petrol/LPG": "Other",
    "Natural gas type H": "Other",
    "LNG": "Other",
    "Diesel/LNG": "Other"
})
traficom2 = traficom2.groupby(["Region", "Vehicle class", "year", "quarter", "Driving power"])["Vehicles in traffic"].sum().reset_index()

# The dataset contains data for the year 2024, quarter 3 and 4, which are not yet complete so dropping them

traficom2 = traficom2[~((traficom2['year'] == 2024) & (traficom2['quarter'].isin([3, 4])))]

# Add a new column that holds the region ID (helps in the analysis phase)

traficom2['region_id'] = traficom2['Region'].apply(lambda region: helpers.get_rid_by_name(region, mappings))

traficom2.head(5)

,Region,Vehicle class,year,quarter,Driving power,Vehicles in traffic,region_id
0,Central Finland,Passenger cars,2008,1,Diesel,21511,13
1,Central Finland,Passenger cars,2008,1,Electricity,0,13
2,Central Finland,Passenger cars,2008,1,Natural gas (CNG),5,13
3,Central Finland,Passenger cars,2008,1,Other,8,13
4,Central Finland,Passenger cars,2008,1,Petrol,103518,13


In [10]:
# Write clean data to a folder containing preprocessed files

helpers.save_dataset(f"{clean_dir}vehicle_class_statistics.csv", traficom2)

## 1c. Electric vehicles in traffic by municipality (kunta) as of 30.9.2024

This dataset contains a snapshot of electric cars per municipality.

- Data source: [Finnish Transport and Communications Agency, Traficom](https://www.traficom.fi/en)
- License: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/deed.en)
- Link: https://trafi2.stat.fi/PXWeb/pxweb/en/TraFi/TraFi__Liikennekaytossa_olevat_ajoneuvot/010_kanta_tau_101.px/

Manual preprocessing steps:

1. Go to https://trafi2.stat.fi/PXWeb/pxweb/en/TraFi/TraFi__Liikennekaytossa_olevat_ajoneuvot/010_kanta_tau_101.px/
2. Select fields:
    - Area: from "Akaa" to "Äänekoski" (293 in total)
    - Make: "Passenger cars total"
    - Year of first registration: "Total"
    - Driving power: "Total", "Electricity", "Petrol/Electricity (plug-in hybrid)", "Diesel/Electricity (plug-in hybrid)"
3. Click "Save result as...", then "Comma delimited with heading" and press "Save", choose "datasets/traficom" as the folder
4. Check the file name matches the one in the next cell

This saved query should result in the same outcome: https://trafi2.stat.fi:443/PXWeb/sq/f201f38a-f198-4688-9209-7b26a19e96db

In [11]:
# Both URLs or file paths are accepted

datafile = f"{dirty_dir}traficom/kanta_2024-09-30.csv"
traficom3 = pd.read_csv(datafile, header=1, sep=",", encoding="ISO-8859-1")

traficom3.head(5)

,Area,Make,Total Total,Total Electricity,Total Petrol/Electricity (plug-in hybrid),Total Diesel/Electricity (plug-in hybrid)
0,Akaa,Passenger cars total,10092,318,319,25
1,Alajärvi,Passenger cars total,5587,84,102,14
2,Alavieska,Passenger cars total,1493,24,38,3
3,Alavus,Passenger cars total,6849,86,125,7
4,Asikkala,Passenger cars total,4905,114,214,12


In [12]:
# Replace missing values with zeroes

traficom3["Total Total"] = traficom3["Total Total"].replace("-", 0)
traficom3["Total Electricity"] = traficom3["Total Electricity"].replace("-", 0)
traficom3["Total Petrol/Electricity (plug-in hybrid)"] = traficom3["Total Petrol/Electricity (plug-in hybrid)"].replace("-", 0)
traficom3["Total Diesel/Electricity (plug-in hybrid)"] = traficom3["Total Diesel/Electricity (plug-in hybrid)"].replace("-", 0)

# Convert columns to numeric

traficom3["Total Total"] = pd.to_numeric(traficom3["Total Total"])
traficom3["Total Electricity"] = pd.to_numeric(traficom3["Total Electricity"])
traficom3["Total Petrol/Electricity (plug-in hybrid)"] = pd.to_numeric(traficom3["Total Petrol/Electricity (plug-in hybrid)"])
traficom3["Total Diesel/Electricity (plug-in hybrid)"] = pd.to_numeric(traficom3["Total Diesel/Electricity (plug-in hybrid)"])

# Precalculate the share of different vehicle types

traficom3["Hybrid Share (%)"] = (traficom3["Total Petrol/Electricity (plug-in hybrid)"] + traficom3["Total Diesel/Electricity (plug-in hybrid)"]) / traficom3["Total Total"] * 100
traficom3["EV Share (%)"] = traficom3["Total Electricity"] / traficom3["Total Total"] * 100

# Add columns holding the region and municipality IDs

traficom3["municipality_id"] = traficom3.apply(lambda row: helpers.get_mid_by_name(row["Area"], mappings), axis=1)
traficom3["region_id"] = traficom3["municipality_id"].apply(lambda muni: helpers.get_rid_by_mid(muni, mappings))

# Rename the column "Area" to "Municipality"

traficom3 = traficom3.rename(columns={"Area": "Municipality"})

traficom3.head(5)

,Municipality,Make,Total Total,Total Electricity,Total Petrol/Electricity (plug-in hybrid),Total Diesel/Electricity (plug-in hybrid),Hybrid Share (%),EV Share (%),municipality_id,region_id
0,Akaa,Passenger cars total,10092,318,319,25,3.408641,3.151011,20,6
1,Alajärvi,Passenger cars total,5587,84,102,14,2.076248,1.503490,5,14
2,Alavieska,Passenger cars total,1493,24,38,3,2.746149,1.607502,9,17
3,Alavus,Passenger cars total,6849,86,125,7,1.927289,1.255658,10,14
4,Asikkala,Passenger cars total,4905,114,214,12,4.607543,2.324159,16,7


In [13]:
# Write clean data to a folder containing preprocessed files

helpers.save_dataset(clean_dir + "electric_vehicles_2024-09-30.csv", traficom3)

## 2. Infrastructure

This dataset is built from the best EV charging map application in Finland.

- Data source: [Sähköautoilijat ry](https://sahkoautoilijat.fi/)
- Link: [Latauskartta](https://latauskartta.fi/)

Manual preprocessing steps: None. The data is extracted from the web application.

In [14]:
# This dataset is updated almost daily so always downloading the latest

datafile = "https://latauskartta.fi/backend/actions.php?action=getData"

# The JSON data is split into three parts: locations, operators, and chargers
# We'll process them separately, then combine locations and chargers

data = requests.get(datafile).json()

In [15]:
# Preprocess chargers

chargers = pd.DataFrame(data["msg"]["chargers"])

# Define data types

chargers["kpl"] = pd.to_numeric(chargers["kpl"])
chargers["kw"] = pd.to_numeric(chargers["kw"])
chargers["kw_total"] = pd.to_numeric(chargers["kw_total"])

# Calculating the total charging capacity in a simplistic way
# In real life chargers share the capacity, but we do not have that data for all chargers

chargers['kw_total'] = chargers.apply(
    lambda row: int(row['kw'] * row['kpl']) if pd.isna(row['kw_total']) or row['kw_total'] == 0 else int(row['kw_total']),
    axis=1
)

# Rename columns

chargers.rename(columns={"location": "location_id", "id": "charger_id"}, inplace=True)

chargers.head(5)

,sublocation_id,charger_id,type,subtype,operator,location_id,updated,status,info,info_en,kpl,kw,kw_total,kw_800v,links
0,None,8234,9,99,9,144,1672339107,active,HUOM! TÄMÄ PISTE EI AVOINNA MUILLE MERKEILLE!,TESLA ONLY,8,150,1200,None,
1,None,8236,9,99,9,152,1662322462,active,"VAIN TESLA, EI AVOIN MUILLE MERKEILLE!",TESLA ONLY,4,150,600,None,
2,None,8235,9,1,9,153,1662322876,active,"VAIN TESLA, EI AVOINNA MUILLE MERKEILLE!",ONLY TESLA,4,150,600,None,
3,None,8232,9,99,9,1,1668545133,active,"VAIN TESLA, EI AVOINNA MUILLE MERKEILLE!",TESLA ONLY,8,120,960,None,
4,None,8233,9,1,9,2,1662322567,active,"VAIN TESLA, EI AVOINNA MUILLE MERKEILLE!",TESLA ONLY,8,150,1200,None,


In [16]:
# Preprocess locations

# If sublocations are encountered, new rows will be created

locations_dict = data["msg"]["locations"]
flattened_data = []
for location_id, location_data in locations_dict.items():
    if "sublocations" in location_data and location_data["sublocations"]:
        for sub_id, sublocation_data in location_data["sublocations"].items():
            sublocation_data["sublocation_id"] = int(sublocation_data["sublocation_id"])
            combined_data = {**location_data, **sublocation_data}
            combined_data.pop("sublocations", None)
            flattened_data.append(combined_data)
    else:
        flattened_data.append(location_data)
locations = pd.DataFrame(flattened_data)

# Define datatypes

locations["lat"] = pd.to_numeric(locations["lat"])
locations["lon"] = pd.to_numeric(locations["lon"])

# Remove whitespace and newlines from the text fields

locations["info"] = locations["info"].str.replace("\n", " ", regex=True).str.replace("\r", "", regex=True)
locations["info_en"] = locations["info_en"].str.replace("\n", " ", regex=True).str.replace("\r", "", regex=True)

# Add columns holding the region and municipality IDs

locations["municipality_id"] = locations.apply(lambda row: helpers.get_mid_by_coords([(row["lon"], row["lat"])], municipality_map), axis=1)
locations["region_id"] = locations["municipality_id"].apply(lambda muni: helpers.get_rid_by_mid(muni, mappings))

locations.head(5)

,id,lat,lon,title,address,info,info_en,status,country,tags,thumb,updated,sublocation_id,title_fi,title_en,thumb_id,published_ts,municipality_id,region_id
0,1,60.443001,22.605499,Paimio Supercharger,"Länsiportti 1, 21530 Paimio",ABC Tammisilta suljettu Ajanvietteenä aseman v...,On the parking lot next to the highway. Cloth...,0,FI,[],49,1687382398,NaN,NaN,NaN,NaN,NaN,577,2
1,2,61.181229,23.884975,ABC Akaa Toijala,"Sirkesalontie 2, 37800 Akaa",Laturit sijaitsevat aseman pohjoispuolella rek...,"On the north side, close to truck parking http...",0,FI,"[2, 4, 9]",11298,1718185183,NaN,NaN,NaN,NaN,NaN,20,6
2,3,60.209702,24.755600,S-market Nihtisilta,"Kutojantie 1, 02630 Espoo",Laturit sijaitsevat S-marketin parkkihallissa....,In the garage. Note! Check the low clearance ...,2,FI,"[2, 4]",5006,1721919996,NaN,NaN,NaN,NaN,NaN,49,1
3,4,60.166029,24.732531,Prisma Olari,"Komeetankatu 2, 02210 Espoo",Laturit sijaitsevat Prisman parkkihallissa (al...,Garage accessible from Kuitinmäentie. https:/...,2,FI,"[2, 4, 5, 10]",12113,1729081595,NaN,NaN,NaN,NaN,NaN,49,1
4,5,63.713770,25.918518,Vaskikello,"Vaskikellontie 420, 86800 Pyhäjärvi",http://www.vaskikello.fi/,They also have a large collection of cowbells....,0,FI,"[2, 7, 9]",10106,1695296704,NaN,NaN,NaN,NaN,NaN,626,17


In [17]:
# Operators are not currently used in the analysis phase so we'll just flatten the data

operators = data["msg"]["operators"]
flattened_data = []
for operator_id, operator_data in operators.items():
    flattened_data.append(operator_data)
operators = pd.DataFrame(flattened_data)

operators.head(5)

,profileFB,profileTwitter,id,name,webpage,classname,priority,sortorder,logo_dark,logo_light,shortname
0,,,27,ABC,https://www.abcasemat.fi/sahkoauton-lataus,abc,1,98,abc/abc_dark.svg,abc/abc_light.svg,slataus
1,,,44,Allego,https://www.allego.eu,None,0,0,allego/allego_logo_color_nobg.svg,allego/allego_logo_color_nobg.svg,None
2,,,40,Autoliitto,https://www.autoliitto.fi/jasenedut/autoliiton...,,1,0,autoliitto/logo.svg,autoliitto/logo.svg,autoliitto
3,,,35,Autopay,https://autopay.io,None,0,0,autopay/autopay_dark.svg,autopay/autopay_light.svg,None
4,,,14,autoPilDYK,,muu,-1,0,None,None,None


In [18]:
# Write clean data to a folder containing preprocessed files

helpers.save_dataset(clean_dir + "charging_locations.csv", locations)
helpers.save_dataset(clean_dir + "charging_operators.csv", operators)
helpers.save_dataset(clean_dir + "charging_chargers.csv", chargers)

## 3. Socioedemographics

This dataset contains a plenty of sociodemographics for municipalities (kunta) or regions (maakunta).

- Data source: [Statistics Finland](https://stat.fi/index_en.html)
- License: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/deed.en)
- Links:
    - [Data](https://pxdata.stat.fi/PxWeb/pxweb/en/Kuntien_avainluvut/Kuntien_avainluvut__uusin/142h.px/)

The extraction tool has some limitations so the municipality data needs to be retrieved in batches.

I'm sure there is a plenty of irrelevant data downloaded, but I rather download/process everything and then assess the relevancy in the Exploratory Data Analysis phase than make guesses at this point.

Below you can find saved queries but you can also go and query the data manually:

Data settings:
- Information: All fields selected
- Region:
    1. Municipalities (kunnat): Akaa-Äänekoski (in batches)
    2. The whole country and regions (maakunnat): WHOLE COUNTRY + MK01-MK21 (1+19 in total)
- Year: 2008-

Saved queries to download the municipality data:
- Akaa-Jyväskylä: https://pxdata.stat.fi:443/PxWeb/sq/e638a573-b854-47d0-aaba-a3a787c08ecd
- Jämijärvi-Lapinlahti: https://pxdata.stat.fi:443/PxWeb/sq/3565bd16-2012-4302-856e-e327a2a9bbd0
- Lappajärvi-Pertunmaa: https://pxdata.stat.fi:443/PxWeb/sq/65c7cd1d-d81a-414f-a57f-809dc1131945
- Petäjävesi-Sottunga: https://pxdata.stat.fi:443/PxWeb/sq/cb15da79-70b8-4a78-b65e-6aa37d587489
- Sulkava-Äänekoski: https://pxdata.stat.fi:443/PxWeb/sq/d613e8bf-707e-457c-8598-6cc573a2d83e

This saved query can be used to download the data for the WHOLE COUNTRY and regions (maakunnat): https://pxdata.stat.fi:443/PxWeb/sq/eb9765dd-f9d7-4416-9650-e63ca58864bd


In [19]:
datafiles = [dirty_dir + "statistics_finland/socio_akaa-jyväskylä.csv",
             dirty_dir + "statistics_finland/socio_jämijärvi-lapinlahti.csv",
             dirty_dir + "statistics_finland/socio_lappajärvi-pertunmaa.csv",
             dirty_dir + "statistics_finland/socio_petäjävesi-sottunga.csv",
             dirty_dir + "statistics_finland/socio_sulkava-äänekoski.csv"]

statfin2 = pd.DataFrame()

# Read datafiles in one by one, concatenate
for datafile in datafiles:
    temp = pd.read_csv(datafile, header=1, sep=",", encoding="ISO-8859-1")
    statfin2 = pd.concat([statfin2, temp], ignore_index=True, copy=True)

statfin2.head(5)

,Region,Information,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Akaa,"Land area, km2",.,.,.,.,.,.,.,293.2,293.2,293.1,293.3,293.3,293.3,293.3,293.3,293.3
1,Akaa,Population with known coordinates,16716,16737,16886,16948,17004,16939,16915,16895,16775,16621,16446,16318,16224,16310,16299,.
2,Akaa,Population,16837,16858,17012,17091,17134,17108,17052,17043,16923,16769,16611,16475,16391,16467,16473,16405
3,Akaa,Mean population,16787.5,16847.5,16935.0,17051.5,17112.5,17121.0,17080.0,17047.5,16983.0,16846.0,16690.0,16543.0,16433.0,16429.0,16470.0,16439.0
4,Akaa,Population increase,95,20,159,78,41,-30,-57,-11,-115,-155,-161,-135,-86,74,4,-67


In [20]:
# Rename column "Region" to "Municipality"

statfin2 = statfin2.rename(columns={"Region": "Municipality"})

# Remove all Information categories that contain values we can't compare between municipalities

statfin2 = statfin2[(statfin2['Information'].str.contains('%', na=False)) |
                    (statfin2['Information'].str.startswith('Average', na=False)) | 
                    (statfin2['Information'].str.endswith('inhabitants', na=False)) |
                    (statfin2['Information'].str.endswith('1,000 population', na=False))]

# Imputation of missing values
# 1. Try interpolation
# 2. Try forward- and backfilling
# 3. Fill with the mean of the column (national mean)

statfin2.replace(".", pd.NA, inplace=True)
columns_to_interpolate = [year for year in statfin2.columns if year.isdigit()]
statfin2[columns_to_interpolate] = statfin2[columns_to_interpolate].apply(pd.to_numeric)
statfin2[columns_to_interpolate] = statfin2[columns_to_interpolate].interpolate(method="linear", axis=1)
statfin2[columns_to_interpolate] = statfin2[columns_to_interpolate].ffill(axis=1).bfill(axis=1)
statfin2[columns_to_interpolate] = statfin2.groupby('Information')[columns_to_interpolate].transform(lambda x: x.fillna(x.mean()))

# Add columns holding the region and municipality IDs

statfin2["municipality_id"] = statfin2.apply(lambda row: helpers.get_mid_by_name(row["Municipality"], mappings), axis=1)
statfin2["region_id"] = statfin2["municipality_id"].apply(lambda muni: helpers.get_rid_by_mid(muni, mappings))

# Remove Åland municipalities from the dataset because the data is not complete :(

statfin2 = statfin2[statfin2['region_id'] != 21]

statfin2.head(5)

,Municipality,Information,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,municipality_id,region_id
6,Akaa,"Population change from the previous year, %",0.6,0.1,0.9,0.5,0.3,-0.2,-0.3,-0.1,-0.7,-0.9,-0.9,-0.8,-0.5,0.5,0.0,-0.4,20,6
7,Akaa,"Population change in five years, %",3.5,3.3,3.6,3.0,2.4,1.6,1.2,0.2,-1.0,-2.1,-2.9,-3.4,-3.8,-2.7,-1.8,-1.2,20,6
12,Akaa,"Persons aged under 7, %",9.0,9.0,9.2,8.9,8.7,8.7,8.4,8.1,7.8,7.4,6.9,6.5,6.1,5.9,5.7,5.6,20,6
14,Akaa,"Persons aged under 15, %",18.9,18.7,19.0,18.8,18.7,18.8,18.6,18.5,18.3,18.0,17.7,17.2,16.5,16.3,15.7,15.3,20,6
15,Akaa,Change in the number of persons aged under 15 ...,0.8,-0.7,2.4,-0.5,-0.2,0.1,-1.1,-0.6,-1.7,-2.9,-2.1,-3.7,-4.5,-1.2,-3.5,-2.9,20,6


In [21]:
# Write clean data to a folder containing preprocessed files

helpers.save_dataset(clean_dir + "socio_municipalities.csv", statfin2)

The same for the whole country + regions (maakunnat).

In [22]:
datafile = f"{dirty_dir}statistics_finland/socio_whole_country_and_regions.csv"

statfin3 = pd.read_csv(datafile, header=1, sep=",", encoding="ISO-8859-1")

statfin3.head(5)

,Region,Information,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,WHOLE COUNTRY,"Land area, km2",.,.,.,.,.,.,.,303900.0,303912.3,303918.5,303921.1,303932.7,303935.2,303947.8,303963.3,303985.4
1,WHOLE COUNTRY,Population with known coordinates,5271448,5295038,5316759,5341034,5363435,5358090,5407760,5422194,5432778,5441906,5447648,5455832,5469715,5480966,5494750,.
2,WHOLE COUNTRY,Population,5326314,5351427,5375276,5401267,5426674,5451270,5471753,5487308,5503297,5513130,5517919,5525292,5533793,5548241,5563970,5603851
3,WHOLE COUNTRY,Mean population,5313399.0,5338870.5,5363351.5,5388271.5,5413970.5,5438972.0,5461511.5,5479530.5,5495302.5,5508213.5,5515524.5,5521605.5,5529542.5,5541017.0,5556105.5,5583910.5
4,WHOLE COUNTRY,Population increase,25893,25095,23824,26197,25219,24710,21067,15421,15714,11423,5015,7159,8789,14840,16095,39958


In [23]:
# Remove all Information categories that contain values we can't compare between regions

statfin3 = statfin3[(statfin3['Information'].str.contains('%', na=False)) |
                    (statfin3['Information'].str.startswith('Average', na=False)) | 
                    (statfin3['Information'].str.endswith('inhabitants', na=False)) |
                    (statfin3['Information'].str.endswith('1,000 population', na=False))]

# Convert the region ID to a purely numeric value

statfin3["region_id"] = pd.to_numeric(statfin3["Region"].str.extract(r"^MK([0-9]{2}) ")[0], errors="coerce").fillna(0).astype(int)
statfin3["Region"] = statfin3["Region"].str.replace(r"^MK[0-9]{2} ", "", regex=True)

# Imputation of missing values
# 1. Try interpolation
# 2. Try forward- and backfilling
# 3. Fill with the mean of the column (national mean)

statfin3.replace(".", pd.NA, inplace=True)
columns_to_interpolate = [year for year in statfin3.columns if year.isdigit()]
statfin3[columns_to_interpolate] = statfin3[columns_to_interpolate].apply(pd.to_numeric)
statfin3[columns_to_interpolate] = statfin3[columns_to_interpolate].interpolate(method='linear', axis=1)
statfin3[columns_to_interpolate] = statfin3[columns_to_interpolate].ffill(axis=1).bfill(axis=1)
statfin3[columns_to_interpolate] = statfin3.groupby('Information')[columns_to_interpolate].transform(lambda x: x.fillna(x.mean()))

# Remove Åland municipalities from the dataset because the data is not complete :(

statfin3 = statfin3[statfin3['region_id'] != 21]

statfin3.head(5)

,Region,Information,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,region_id
6,WHOLE COUNTRY,"Population change from the previous year, %",0.5,0.5,0.4,0.5,0.5,0.5,0.4,0.3,0.3,0.2,0.1,0.1,0.2,0.3,0.3,0.7,0
7,WHOLE COUNTRY,"Population change in five years, %",2.0,2.2,2.3,2.4,2.4,2.3,2.2,2.1,1.9,1.6,1.2,1.0,0.8,0.8,0.9,1.6,0
12,WHOLE COUNTRY,"Persons aged under 7, %",7.7,7.7,7.8,7.8,7.8,7.8,7.7,7.6,7.5,7.3,7.0,6.8,6.6,6.4,6.2,6.0,0
14,WHOLE COUNTRY,"Persons aged under 15, %",16.7,16.6,16.5,16.5,16.4,16.4,16.4,16.3,16.2,16.2,16.0,15.8,15.6,15.4,15.1,14.9,0
15,WHOLE COUNTRY,Change in the number of persons aged under 15 ...,-0.4,-0.3,-0.1,0.1,0.3,0.4,0.2,-0.1,-0.2,-0.4,-0.9,-1.3,-1.2,-1.1,-1.3,-1.0,0


In [24]:
# Write clean data to a folder containing preprocessed files.

helpers.save_dataset(clean_dir + "socio_whole_country_and_regions.csv", statfin3)

## 4. Income

**Year 2023 data was released during the mini project. To be updated.**

This dataset contains various tax data for municipalities (kunta).

- Data source: [Tax Administration](https://vero.fi/en/)
- License: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/deed.en)
- Links:
    - [Data](https://vero2.stat.fi/PXWeb/pxweb/en/Vero/Vero__Henkiloasiakkaiden_tuloverot__lopulliset__alue/tulot_102.px/)

The data needs to be retrieved in batches.

I'm sure there is a plenty of irrelevant data, but I rather wanted to conclude the relevancy in the Exploratory Data Analysis phase than make guesses.

Fields:
- Statistic: all
- Region:
    - Country: the first one on the list
    - Regions (maakunnat): 2-20 on the list, from Uusimaa to Åland
    - Municipalities: Akaa-Äänekoski
- Tax year: 2014-2022
- Variable: all

Manual pivot:
- Rows: Region, Variable, Statistic
- Columns: Tax year

Municipalities
- Akaa-Järvenpää (https://vero2.stat.fi:443/PXWeb/sq/8d355db7-85e0-4cf7-a1bf-8d52d1befa9e)
- Kaarina-Lumijoki (https://vero2.stat.fi:443/PXWeb/sq/606a5cdd-394f-45a2-b62e-14eac9245b37)
- Lumparland-Reisjärvi (https://vero2.stat.fi:443/PXWeb/sq/17956dd1-043c-4160-bd7e-0c4090f5f47e)
- Riihimäki-Äänekoski (https://vero2.stat.fi:443/PXWeb/sq/c9130ed8-fc94-4cb6-9c84-8bf7dad16313)

WHOLE COUNTRY + maakunnat
https://vero2.stat.fi:443/PXWeb/sq/9df726c6-6cda-42d4-91bd-06427dcb50b6

In [25]:
# Read in all batches and merge into one dataframe

datafiles = [dirty_dir + "tax_authority/income_akaa-järvenpää.csv",
             dirty_dir + "tax_authority/income_kaarina-lumijoki.csv",
             dirty_dir + "tax_authority/income_lumparland-reisjärvi.csv",
             dirty_dir + "tax_authority/income_riihimäki-äänekoski.csv"]

tax1 = pd.DataFrame()

for datafile in datafiles:
    temp = pd.read_csv(datafile, header=1, sep=",", encoding="ISO-8859-1")
    tax1 = pd.concat([tax1, temp], ignore_index=True, copy=True)

tax1.head(5)

,Region,Variable,Statistic,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Akaa,1. Quantity of recipients of Tax Decisions,"Sum, euros",-,-,-,-,-,-,-,-,-
1,Akaa,1. Quantity of recipients of Tax Decisions,Number,15496,15604,15558,15518,15382,15281,15219,15204,15386
2,Akaa,1. Quantity of recipients of Tax Decisions,Mean,-,-,-,-,-,-,-,-,-
3,Akaa,1. Quantity of recipients of Tax Decisions,Lower quartile,-,-,-,-,-,-,-,-,-
4,Akaa,1. Quantity of recipients of Tax Decisions,Upper quartile,-,-,-,-,-,-,-,-,-


In [26]:
# Hide warnings

pd.set_option("future.no_silent_downcasting", True)

# Remove data from abolished municipalities

tax1 = tax1[~tax1["Region"].str.contains("abolished", case=False, na=False)]

# Iterate the data municipality by municipality for imputation of missing values, precalculations

for municipality in tax1["Region"].unique():
    municipality_data = tax1[tax1["Region"] == municipality].copy()

    # The number of residents is needed for calculating per capita values

    residents_row = municipality_data[
        (municipality_data["Variable"].str.contains("Residents", na=False)) & 
        (municipality_data["Statistic"] == "Number")
    ]

    # Replace missing values

    if not residents_row.empty:
        residents = residents_row.iloc[0, 3:].replace(["-", ".."], 0).astype(float)
        
        if (residents == 0).all():
            print(f"Warning: Invalid resident data for {municipality}.")
            continue

        # Iterate the Information categories

        for index, row in municipality_data.iterrows():

            # Replace missing values with zeroes

            row.iloc[3:] = row.iloc[3:].replace({"-": 0, "..": 0})
            
            # Absolute total sums will be converted to per capita values below

            if "Sum, euros" in row["Statistic"]:
                row["Statistic"] = "Per capita, euros"

            # Convert to float

            row.iloc[3:] = row.iloc[3:].astype(float)  

            # If the data is in percentages, leave it as is

            if "%" in row["Statistic"]:
                continue
            
            # Process "Per capita, euros" variables

            if "Per capita, euros" in row["Statistic"]:
                for year_index in range(9):
                    if row.iloc[year_index + 3] != 0:
                        if residents.iloc[year_index] != 0:
                            row.iloc[year_index + 3] = row.iloc[year_index + 3] / residents.iloc[year_index]

    # Save changes back to the dataframe

    tax1.loc[tax1["Region"] == municipality] = municipality_data

# Set the data types

for year in range(2014, 2023):
    tax1[str(year)] = pd.to_numeric(tax1[str(year)], errors="coerce")

# Rename column "Region" to "Municipality"

tax1.rename(columns={"Region": "Municipality"}, inplace=True)

# Add column holding the municipality and region IDs

tax1["municipality_id"] = tax1.apply(lambda row: helpers.get_mid_by_name(row["Municipality"], mappings), axis=1)
tax1["region_id"] = tax1["municipality_id"].apply(lambda muni: helpers.get_rid_by_mid(muni, mappings))

tax1.head(5)

,Municipality,Variable,Statistic,2014,2015,2016,2017,2018,2019,2020,2021,2022,municipality_id,region_id
0,Akaa,1. Quantity of recipients of Tax Decisions,"Per capita, euros",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,6
1,Akaa,1. Quantity of recipients of Tax Decisions,Number,15496.0,15604.0,15558.0,15518.0,15382.0,15281.0,15219.0,15204.0,15386.0,20,6
2,Akaa,1. Quantity of recipients of Tax Decisions,Mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,6
3,Akaa,1. Quantity of recipients of Tax Decisions,Lower quartile,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,6
4,Akaa,1. Quantity of recipients of Tax Decisions,Upper quartile,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,6


In [27]:
# Write clean data to a folder containing preprocessed files

helpers.save_dataset(clean_dir + "income_municipalities.csv", tax1)

In [28]:
datafile = dirty_dir + "tax_authority/income_whole_country_and_regions.csv"

tax2 = pd.read_csv(datafile, header=1, sep=",", encoding="ISO-8859-1")

tax2.head(5)

,Region,Variable,Statistic,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,WHOLE COUNTRY,1. Quantity of recipients of Tax Decisions,"Sum, euros",-,-,-,-,-,-,-,-,-
1,WHOLE COUNTRY,1. Quantity of recipients of Tax Decisions,Number,5148496,5213504,5212079,5258528,5272642,5281424,5304698,5312290,5392864
2,WHOLE COUNTRY,1. Quantity of recipients of Tax Decisions,Mean,-,-,-,-,-,-,-,-,-
3,WHOLE COUNTRY,1. Quantity of recipients of Tax Decisions,Lower quartile,-,-,-,-,-,-,-,-,-
4,WHOLE COUNTRY,1. Quantity of recipients of Tax Decisions,Upper quartile,-,-,-,-,-,-,-,-,-


In [29]:
# Hide warnings

pd.set_option("future.no_silent_downcasting", True)

# Replace missing values with zeroes

tax2.fillna(0, inplace=True)

# Iterate the data region by region for imputation of missing values, precalculations

for region in tax2["Region"].unique():
    region_data = tax2[tax2["Region"] == region].copy()

    # The number of residents is needed for calculating per capita values

    residents_row = region_data[
        (region_data["Variable"].str.contains("Residents", na=False)) & 
        (region_data["Statistic"] == "Number")
    ]

    # Replace missing values
    
    if not residents_row.empty:
        residents = residents_row.iloc[0, 3:].replace(["-", ".."], 0).astype(float)
        
        if (residents == 0).all():
            print(f"Warning: Invalid resident data for {region}.")
            continue

        # Iterate the Information categories

        for index, row in region_data.iterrows():
            
            # Replace missing values

            region_data.loc[index, region_data.columns[3:]] = row.iloc[3:].replace({"-": 0, "..": 0}).astype(float)
            
            # Absolute total sums will be converted to per capita values below

            if "Sum, euros" in row["Statistic"]:
                region_data.at[index, "Statistic"] = "Per capita, euros"

            # If the data is in percentages, leave it as is

            if "%" in row["Statistic"]:
                continue
            
            # Process "Per capita, euros" variables

            if "Per capita, euros" in row["Statistic"]:
                for year_index in range(9):
                    year_col_index = year_index + 3
                    if year_col_index < len(region_data.columns):
                        if row.iloc[year_col_index] != 0 and residents.iloc[year_index] != 0:
                            region_data.loc[index, region_data.columns[year_col_index]] = (
                                row.iloc[year_col_index] / residents.iloc[year_index]
                            )

    # Save changes back to the dataframe

    tax2.loc[tax2["Region"] == region, :] = region_data

# Set the data types

for year in range(2014, 2023):
    tax2[str(year)] = pd.to_numeric(tax2[str(year)], errors="coerce")

# Add column holding the region IDs

tax2["region_id"] = tax2["Region"].apply(lambda region: helpers.get_rid_by_name(region, mappings))

# Remove Åland from the dataset because the data is not complete :(

tax2 = tax2[tax2["region_id"] != 21]

tax2.head(5)

,Region,Variable,Statistic,2014,2015,2016,2017,2018,2019,2020,2021,2022,region_id
0,WHOLE COUNTRY,1. Quantity of recipients of Tax Decisions,"Per capita, euros",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,WHOLE COUNTRY,1. Quantity of recipients of Tax Decisions,Number,5148496.0,5213504.0,5212079.0,5258528.0,5272642.0,5281424.0,5304698.0,5312290.0,5392864.0,0
2,WHOLE COUNTRY,1. Quantity of recipients of Tax Decisions,Mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,WHOLE COUNTRY,1. Quantity of recipients of Tax Decisions,Lower quartile,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,WHOLE COUNTRY,1. Quantity of recipients of Tax Decisions,Upper quartile,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [30]:
# Save clean data to a folder containing preprocessed files

helpers.save_dataset(clean_dir + "income_whole_country_and_regions.csv", tax2)

That's all for preprocessing for now...